# MICCAI 2023 Tutorial - Part I:
## Launch a containerized medical imaging application
This Notebook is designed to reproduce the computational experiments of a [published scientific paper](https://hal.science/hal-04006057) on deep learning for **Bra**in **T**umor **S**egmentation (**BraTS**).

<img src="imgs/BraTS-Results.png" alt="BraTS-Results" height="250" title="Results from Brain Tumor Segmentation"/>

It uses methods from the [VIP client for Python](https://github.com/virtual-imaging-platform/VIP-python-client).

>__N.B.__: This tutorial requires inputs from the user. Please carefully read the instructions provided by the *italic-style cell comments*.

---
## Useful libraries

Install: *Uncomment & run the following cell if you are using Google Colab*

In [ ]:
# Remove the hashtag (#) only
#! pip install vip-client girder-client

Import

In [ ]:
# Builtins
from pathlib import *
# Third-party
from vip_client.classes import VipLauncher
from vip_client.classes import VipLoader

---
## Prepare your executions

Handshake with VIP using your **API key** 

*You can follow this [procedure](../../doc/account.md) to get a VIP API key from the [VIP portal](https://vip.creatis.insa-lyon.fr/).*

In [ ]:
# Paste your VIP API key between the quotes
VipLauncher.init(api_key="VIP_API_KEY");

To provide some variability in the execution environment, this pipeline will be launched by different __teams__.

*Please wait for the speakers to make the teams & put your __team number__ instead of `None` in the following cell.*

In [ ]:
# PUT YOUR TEAM NUMBER
MY_TEAM_NBR = None

---
## Launch the Brain Tumor Segmentation

### Dataset
Show the (small) dataset that will be used in this tutorial

In [ ]:
# The data you need is stored in a folder on VIP Servers 
input_dir = PurePosixPath("/vip/MICCAI tutorial (group)/inputs")
# This folder contains data from several subjects
subjects = VipLoader.list_dir(input_dir)
print("\n".join(subjects))

### Software
The `BraTS` pipeline is declined in **two different versions** that require the same inputs.

In [ ]:
# Available versions
pipelines = {
    "v181": "BraTSPipeline/1.8.1_miccai", 
    "v190": "BraTSPipeline/1.9.0_miccai"
}
# Show the inputs required by the pipeline
VipLauncher.show_pipeline(pipelines["v181"])

### Execution Environment
Your team will launch BraTS with a fixed number of threads for parallel computing.

In [ ]:
# (raises ValueError if no team number is provided)
try:
    # Select a number of threads based on the team number
    nb_threads = 8 if (MY_TEAM_NBR % 2 == 0) else 16
except TypeError:
    raise ValueError("Please precise your team number in the cell above")

### Application Launch
*Please execute the following cell __only once__ and go to Notebook N°2.*

In [ ]:
# Each subject contains 4 inputs files for the BraTS pipeline
input_settings = {
    "t1Image": [(input_dir / subject / "T1.nii.gz") for subject in subjects],
    "t1ceImage": [(input_dir / subject / "T1GD.nii.gz") for subject in subjects],
    "t2Image": [(input_dir / subject / "T2.nii.gz") for subject in subjects],
    "flImage": [(input_dir / subject / "T2-FLAIR.nii.gz") for subject in subjects],
    "appliOutputDir": subjects,
    "brainTumor": "1",
    "skullStrip": "1",
    "bundleAllResults": "0",
    "nb_threads": str(nb_threads)
}

# Launch both versions of the application
for version in pipelines:
    session = VipLauncher(
        output_dir = input_dir.parent / "outputs" / ("miccai-team-"+str(MY_TEAM_NBR)) / version,
        pipeline_id = pipelines[version],
        input_settings = input_settings
    ).launch_pipeline();
session.monitor_workflows();

You should be invited to monitor your executions on the [VIP portal](https://vip.creatis.insa-lyon.fr).

After the coffee break, please proceed to the **[Hands-On Part II](2-analyze-outputs.ipynb)**.

---